In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:

pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df['Country Code']=pop_df['Country Code'].apply(lambda x: x.lower())
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values

excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    "GEO", #Georgia
    'SWZ', 
    'PRK', #North Korea
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]


possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

In [3]:

df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df.rename(columns={'aggregated_value': 'count', 'country': 'Mention_country', 'affiliation_country': 'Aff_country'}, inplace=True)
df=df[(df['Mention_country'].isin(possible_iso))&(df['Aff_country'].isin(possible_iso))*(df['Aff_country']!=df['Mention_country'])]
df = df[df['year'].isin(np.arange(2002, 2020))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
df=df[~df['subjarea'].isin(physical_sciences)]
df=df.groupby(['year', 'Mention_country'])['count'].sum().reset_index()


data=pd.read_csv(r"C:\Users\Yasaman\Downloads\scopus_2024_V1_scholarlymigration_country_enriched.csv")
data=data[data['year'].isin(np.arange(2002, 2020))]
data=data[['iso3code', 'incomelevel', 'gdp_per_capita', 'year', 'population', 'region', 'padded_population_of_researchers']].dropna()
data.rename(columns={'iso3code':'Mention_country'}, inplace=True)
data['Mention_country']=data['Mention_country'].apply(lambda x: x.lower())
df=df.merge(data, on=['Mention_country', 'year'], how='outer')
df=df[df['Mention_country'].isin(possible_iso)]


countries_to_remove=[]
for c  in df['Mention_country'].unique():
    if ((~df['count'].isna()) & (df['Mention_country'] == c)).sum()<15:
        countries_to_remove.append(c)
        print(c)

print(len(countries_to_remove))


# Define the required year range
required_years = list(range(2002, 2020))

# Get the unique countries
unique_countries = df["Mention_country"].unique()

# Create a complete DataFrame with all country-year combinations
full_data = []
for country in unique_countries:
    country_data = df[df["Mention_country"] == country]
    existing_years = set(country_data["year"])
    
    for year in required_years:
        if year in existing_years:
            row = country_data[country_data["year"] == year].iloc[0].to_dict()
        else:
            row = {
                "year": year,
                "Mention_country": country,
                "count": 0,
                "gdp_per_capita": np.nan,
                "population": np.nan,
                "region": country_data["region"].iloc[0] if not country_data.empty else np.nan,
            }
        full_data.append(row)

# Convert to DataFrame
df_complete = pd.DataFrame(full_data)

df_complete['treated']=df_complete['Mention_country'].isin(abbr).astype(int)
df_complete['treated_CW']=df_complete['Mention_country'].isin(['yem', 'lby', 'syr']).astype(int)
df_complete['treated_GO']=df_complete['Mention_country'].isin(['egy', 'tun']).astype(int)
df_complete['treated_GC']=df_complete['Mention_country'].isin(['omn', 'kwt', 'bhr', 'mar','jor']).astype(int)
df_complete['post']=df_complete['year'].apply(lambda x: 0 if x>=2002 and x<=2010 else 1 )
df_complete['count']=df_complete['count'].fillna(0)
df_complete['log_count']=np.log(df_complete['count']+1)

df_complete[['region', 'gdp_per_capita', 'population','padded_population_of_researchers']] = df_complete.groupby('Mention_country')[[ 'region', 'gdp_per_capita', 'population','padded_population_of_researchers']].ffill()
df_complete[[ 'region', 'gdp_per_capita', 'population','padded_population_of_researchers']] = df_complete.groupby('Mention_country')[[ 'region', 'gdp_per_capita', 'population','padded_population_of_researchers']].bfill()
df_complete['log_gdp']=np.log(df_complete['gdp_per_capita'])
df_complete['log_population']=np.log(df_complete['population'])
df_complete['log_Rpop']=np.log(df_complete['padded_population_of_researchers']+1)
df_complete=df_complete[df_complete['Mention_country'].isin(possible_iso)].reset_index(drop=True)


C:\Users\Yasaman\AppData\Local\Temp\ipykernel_26876\290222154.py:3: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  df=df[(df['Mention_country'].isin(possible_iso))&(df['Aff_country'].isin(possible_iso))*(df['Aff_country']!=df['Mention_country'])]


0


In [4]:
def find_match( country_list, n_match_per_country, variables, dataset): 
    matched_countries = []
    for country in country_list:
        country_data = dataset[(dataset['Mention_country'] == country) &(dataset['year']<=2010)]
        if country_data.empty:
            continue
        country_means = country_data[variables].mean()
        
        # Exclude already treated countries and those already matched
        potential_matches = dataset[
            (~dataset['Mention_country'].isin(country_list)) & 
            (~dataset['Mention_country'].isin(matched_countries))
        ]
        
        potential_matches_means = potential_matches.groupby('Mention_country')[variables].mean()
        possible_possible=list(set(possible_iso)-set(abbr))
        potential_matches_means=potential_matches_means[potential_matches_means.index.isin(possible_possible)]
        # Calculate Euclidean distances
        distances = np.linalg.norm(potential_matches_means - country_means, axis=1)
        
        # Get the top N matches
        top_matches = potential_matches_means.index[np.argsort(distances)[:n_match_per_country]]
        
        matched_countries.extend(top_matches)
    
    return sorted(set(matched_countries))


In [16]:
df_complete


,year,Mention_country,count,incomelevel,gdp_per_capita,population,region,padded_population_of_researchers,treated,treated_CW,treated_GO,treated_GC,post,log_count,log_gdp,log_population,log_Rpop
18,2002.0,ago,27.460718,LMC,872.657638,17516139.0,Sub-Saharan Africa,27.0,0,0,0,0,0,3.348525,6.771543,16.678633,3.332205
19,2003.0,ago,31.119439,LMC,982.805590,18124342.0,Sub-Saharan Africa,38.0,0,0,0,0,0,3.469461,6.890411,16.712766,3.663562
20,2004.0,ago,38.447955,LMC,1254.696126,18771125.0,Sub-Saharan Africa,44.0,0,0,0,0,0,3.674982,7.134649,16.747830,3.806662
21,2005.0,ago,64.893290,LMC,1900.723816,19450959.0,Sub-Saharan Africa,48.0,0,0,0,0,0,4.188037,7.549990,16.783407,3.891820
22,2006.0,ago,46.168818,LMC,2597.963585,20162340.0,Sub-Saharan Africa,57.0,0,0,0,0,0,3.853733,7.862483,16.819327,4.060443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,2015.0,yem,97.801591,LIC,1488.416269,28516545.0,Middle East & North Africa,526.0,1,1,0,0,1,4.593114,7.305468,17.165995,6.267201
2552,2016.0,yem,101.814440,LIC,1069.816998,29274002.0,Middle East & North Africa,557.0,1,1,0,0,1,4.632926,6.975243,17.192210,6.324359
2553,2017.0,yem,105.904865,LIC,893.716494,30034389.0,Middle East & North Africa,580.0,1,1,0,0,1,4.671939,6.795389,17.217854,6.364751
2554,2018.0,yem,106.051550,LIC,701.714869,30790513.0,Middle East & North Africa,639.0,1,1,0,0,1,4.673310,6.553527,17.242717,6.461468


In [5]:
n_counts=5
dataset=df_complete
variables = [ 'log_gdp', 'log_population']
cw_match=find_match(['lby', 'syr', 'yem'], n_counts, variables, dataset)
GO_match=find_match( ['tun', 'egy'], n_counts, variables, dataset)
GC_match=find_match( ['omn', 'kwt', 'bhr', 'mar','jor'], n_counts, variables, dataset)

In [6]:
all_matches=list(set(cw_match+GO_match+GC_match))
print('all matches:', all_matches)

all matches: ['slv', 'vnm', 'ken', 'tkm', 'phl', 'gha', 'lbn', 'blr', 'nzl', 'est', 'gtm', 'hnd', 'qat', 'mus', 'nga', 'tto', 'rou', 'hrv', 'kaz', 'pri', 'cri', 'cyp', 'ury', 'aze', 'ven', 'irq', 'svn', 'gab', 'bgr', 'khm', 'pry', 'ukr', 'pse', 'npl', 'dza', 'mli', 'lva', 'ecu', 'cmr', 'dom', 'bwa', 'lka', 'pan', 'chl', 'mys', 'ltu', 'sgp', 'ago']


In [7]:
df_complete=df_complete[df_complete['Mention_country'].isin(abbr+all_matches)]

In [11]:
df_p=df_complete[df_complete['year']<2011].reset_index(drop=True)
df_p['time']=df_p['year']-2011
# Running the Difference-in-Differences regression
model = smf.ols("log_count ~ time+treated_GO *time+treated_CW *time+treated_GC *time +log_gdp+log_population+ C(Mention_country)", data=df_p).fit(cov_type='cluster', cov_kwds={'groups': df_p['Mention_country']})
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_count   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                     104.6
Date:                Thu, 04 Sep 2025   Prob (F-statistic):           6.56e-29
Time:                        09:36:24   Log-Likelihood:                 251.21
No. Observations:                 522   AIC:                            -374.4
Df Residuals:                     458   BIC:                            -101.9
Df Model:                          63                                         
Covariance Type:              cluster                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 66, but rank is 6
  warnings.warn('covariance of constraints does not have full '


In [14]:
from linearmodels.panel import PanelOLS

panel_data = df_complete.set_index(['Mention_country', 'year'])

model = PanelOLS.from_formula(
    'log_count ~ treated_GO : post+treated_CW : post+treated_GC : post +log_population +log_gdp+ EntityEffects + TimeEffects',
    data=panel_data
)


results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:              log_count   R-squared:                        0.1444
Estimator:                   PanelOLS   R-squared (Between):              0.3310
No. Observations:                1044   R-squared (Within):               0.2871
Date:                Thu, Sep 04 2025   R-squared (Overall):              0.3305
Time:                        09:36:49   Log-likelihood                    388.79
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      32.548
Entities:                          58   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                   F(5,964)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             5.0729
                            

In [15]:
from linearmodels.panel import PanelOLS

panel_data = df_complete.set_index(['Mention_country', 'year'])

model = PanelOLS.from_formula(
    'log_count ~ treated : post +log_population +log_gdp+ EntityEffects + TimeEffects',
    data=panel_data
)

results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:              log_count   R-squared:                        0.0610
Estimator:                   PanelOLS   R-squared (Between):              0.9509
No. Observations:                1044   R-squared (Within):               0.1559
Date:                Thu, Sep 04 2025   R-squared (Overall):              0.9435
Time:                        09:37:05   Log-likelihood                    340.24
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      20.931
Entities:                          58   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                   F(3,966)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             2.5112
                            